# Plotting odometry data

To run this notebook you will need to be working in an environment where pupil_recording_interface and the other packages listed below have been installed. It is suggested to create a new environment called "odometry" following the instructions in the readme.
    -and to install the pupil_recording_interface package first using 'conda install -c vedb pupil_recording_interface'. 

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# Get Session loader from VEDB utils
%run ../../code/vedb_utils.py

import pupil_recording_interface as pri

## Load data

In [ ]:
folder = Path("/home/data/odometry/")
subject = "example_session"

Below, we use pupil_recording_interface to load the data

In [ ]:
ls /home/data/odometry/

In [ ]:
# Load all fields in a list of dictionaries, from 0 to 12 seconds
session = Session(folder=folder / subject)
accel_time, accel_all = session.load("accel", time_idx=(0, 12))
gyro_time, gyro_all = session.load("gyro", time_idx=(0, 12))
odometry_time, odometry_all = session.load('odometry', time_idx=(0, 12))

In [ ]:
# Test run this if you like - big difference in time
accelpri = pri.load_dataset(folder / subject, accel="recording", cache=False)
gyropri = pri.load_dataset(folder / subject, gyro="recording", cache=False)
odometrypri = pri.load_dataset(folder / subject, odometry="recording", cache=False)

Now let's see what we've got loaded. The gyroscope angular velocity data takes the form of an array, the 3 dimensions of angular velocity varying over time. Each time point is loaded, by default, as a dictionary. 

In [ ]:
gyro_all[0]

We can load the data as an array, more amenable to plotting, with the following syntax:

In [ ]:
# Load from zero to 12 seconds
gyro_time, gyro_velocity = session.load("gyro:angular_velocity", time_idx=(0, 12))

In [ ]:
gyro_velocity[:10]

In [ ]:
gyro_time[:10]

The data is recorded at about 200 Hz and it is in units of radians/sec. Let's plot the first 10 seconds of the data (10sec x 200 frames/sec=2000 frames). Time is diplayed as hour:minute:second of the recording. This is a 5 hour recording, more than 3 million samples! Try editing the start and end indices to explore the data. Can you find segments where the activity is obviously different?

In [ ]:
# Plot gyroscope data against time
plt.plot(gyro_time, gyro_velocity)
plt.legend(['x','y','z'])

OK, now let's look at the raw accelerometer data for the same time period (0 to 12 s)

In [ ]:
accel_time, accel_all = session.load("accel", time_idx=(0, 12))

The accelerometer measures linear acceleration

In [ ]:
accel_all[0]

We can load this explicitly as an array in the same way as above: 

In [ ]:
accel_time, accel = session.load("accel:linear_acceleration", time_idx=(0, 12))

In [ ]:
# For the same time steps as the gyroscope, the accelerometer 
# collects less data
accel_time.shape, accel.shape

In [ ]:
accel[:10]

In [ ]:
gyro[:10]

Notice, there are many fewer samples. This is because the accelerometer is only sampling at 60 Hz. We can plot accelerometer data similarly. Measurements are in unti of m/s^2. You can see that the accelerometer is consistently registering gravitational force of close to 9.8 m/s^2 along one or more of the axes. Again, try exploring the data to find segments that are obviously different. What do you think is causing those differences?

In [ ]:
plt.plot(accel_time, accel)
plt.legend(['x','y','z'])

OK, now let's look at the odometry data. These are the estimates that are generated by the Realsense using visual-inertial simulataneous localization and mapping (VI-SLAM). The main advantage is that we get complete estimates of linear and angular position, velocity, and acceleration, each with 3 degrees of freedom.

In [ ]:
odometry_all[0]

All the data is at about 200 Hz, but the number of samples differs slightly from the number for the gyroscope. We can load arrays and plot any of these estimates using the same methods we used for the accelerometer and gyroscope. Let's start with exploring linear velocity, expressed in m/s.

In [ ]:
linear_velocity_time, linear_velocity = session.load('odometry:linear_velocity', time_idx=(151, 252))

In [ ]:
plt.plot(linear_velocity_time, linear_velocity)
plt.legend(['x','y','z'])

I've chosen a time segment where velocity transitions from about 0 m/s to about 1 m/s at about 175s into the recording. What is happening here? Another transition occurs around 220 s. What can account for this transition?

Now we will load one last dataset that is a processed version of the odometry data. It is stored in a NetCDF file, which is convenient for multidimensional time-varying data.

In [ ]:
accel_calib = xr.open_dataset(folder / subject / "acceleration.nc")

This is loaded in a labeled array callex an xarray. Let's see what we've got

In [ ]:
accel_calib

Now let's plot the same time segment for linear velocity that we looked at above (the time axis is different, but this is the same chunk)

In [ ]:
start_ind=6000;
end_ind=10000;
plt.plot(accel_calib.time[start_ind:end_ind],accel_calib.linear_vel[start_ind:end_ind])
plt.legend(['x','y','z'])

What is different in this processed version of the linear velocity data? What "processing" was done?